In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

In [ ]:
sys.path.append(os.path.abspath(os.path.join("../../..")))

In [4]:
import numpy as np
import pandas as pd

from notebooks.utils.constants import question_ids_map
from notebooks.utils.experiment import Experiment
from notebooks.utils.questions import (
    construct_single_question_for_ai,
)

/Users/david/eafit/eafit-tesis/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Threshold for Levenshtein distance


In [5]:
threshold = 80

Retrieve questions


In [ ]:
questions_df = pd.read_excel("../../../data/asc_842/grt_ai_reader_questions_v5.xlsx")
questions_df = questions_df.drop(["encoding"], axis=1)
questions_df = questions_df.replace({np.nan: None})

Retrieve answers file


In [ ]:
answers_df = pd.read_excel("../../../data/asc_842/lease_agreements_info/answers.xlsx")

Get questions to send to the AI


In [8]:
question_id = question_ids_map["lessor_name"]
question_df_filtered = questions_df[questions_df["id"] == question_id]
question_as_json = question_df_filtered.to_dict(orient="records")[0]
question_for_ai = construct_single_question_for_ai(question_as_json)
question_for_ai

{'id': 2,
 'page_reference': 'qualification-and-setup/counter-party-name',
 'display_question': 'What is the name of the lessor?',
 'reader_question': "What is the exact name of the lessor in the source document agreement? Note that the lessor may also be called the 'landlord'.",
 'html_object_id': 'name',
 'restrictions': "Return only the full name of the lessor, formatted in title case if it appears in all caps. Exclude 'landlord' or 'lessor' from your response. Use full names for all entities mentioned.",
 'type': 'string',
 'response_mapping': None,
 'user_response': None}

In [9]:
template_string = """Your task is to read the provided Source Document and answer the questions provided in Inquiries.
    Inquiries is an object representing a question and its parameters. The relevant keys are the following:

    - id: the question's id;
    - reader_question: the question you have to answer;
    - restrictions: any requirements for or restrictions on the response you can provide for that specific question.

    Provide the answer in the "answer_string" key in the form of an array of strings in which each paragraph is a separate string in the array. If the question asks to provide the response in the form of a list,
    then the first string in the array should be an introduction of the list and the remaining strings should be the list. 
    
    In "meta", where possible, you should first quote the sentence where you found this answer. And, if possible, you should explain how you determined your answer.
    
    If your response does not meet the following requirements then fix it:
    - Your answer must come from the provided Source Document. Remove any information from outside the Source Document.
    - Return nothing if there is any type of error, you have no response or you do not know the answer to the question.
    - Confirm that all the restrictions associated with a question are enforced.
    - Do not mention, disclose or reference the restrictions associated with a question anywhere in the response you provide.

    Inquiries: {questions}
    Source Document: {context}
    """

## Experiment 10:

- gpt-4o-mini
- text-embedding-3-large
- structured outputs


In [10]:
experiment = Experiment(
    answers_df=answers_df,
    question_id=question_id,
    question_for_ai=question_for_ai,
    threshold=threshold,
    model="gpt-4o-mini",
    embedding_model="text-embedding-3-large",
    use_structured_outputs=True,
    index_name="eafit-text-embedding-3-large",
    lessor_question=True,
    csv_results_filename="results/lessor_name_prompt_tuning_v5.csv",
    template_string=template_string,
)
experiment.run()

Answer: Kyle J. Beller
Real answer: BarBell Real Estate, LLC
INCORRECT
----------------------------------------------------
Answer: Augmedix Bangladesh Ltd.
Real answer: Sony Chocolate Industries Ltd
INCORRECT
----------------------------------------------------
Answer: Licensor
Real answer: The Wheelership LLC
INCORRECT
----------------------------------------------------
Answer: Venable Tenant, LLC
Real answer: Venable Tenant, LLC
CORRECT
----------------------------------------------------
Answer: Marathon Business
Real answer: Wyomissing Professional Center III Limited Partnership
INCORRECT
----------------------------------------------------
Answer: Lessor
Real answer: Hooten Non Exempt Family Trust B
INCORRECT
----------------------------------------------------
Answer: Landlord
Real answer: AR Industrial No. 1, Ltd
INCORRECT
----------------------------------------------------
Answer: Embassy Bank
Real answer: Village Corner, LLC
INCORRECT
---------------------------------------